In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

# 1. Add headers to mimic a real browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
    # 2. Make request with error handling
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()  # Raises exception for 4XX/5XX errors
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 3. More robust table finding
    tables = soup.find_all('table', class_='historical_data_table')  # Try specific class
    
    if not tables:
        # Alternative table search
        tables = soup.select('table.historical_data_table') or soup.find_all('table')
    
    if not tables:
        raise ValueError("No tables found on the page")
    
    # 4. Find the revenue table
    revenue_table = None
    for table in tables:
        if "Revenue" in str(table):
            revenue_table = table
            break
    
    if revenue_table is None:
        revenue_table = tables[0]  # Fallback to first table
        
    # 5. Process the table
    tesla_revenue = pd.read_html(str(revenue_table))[0]
    
    # 6. Clean the data
    tesla_revenue.columns = ['Date', 'Revenue']
    tesla_revenue['Revenue'] = (
        tesla_revenue['Revenue']
        .str.replace(r'[\$,]', '', regex=True)  # Combined regex replacement
        .astype(float)
    )
    
    print("Last 5 rows of Tesla Revenue:")
    print(tesla_revenue.tail())

except Exception as e:
    print(f"Error occurred: {str(e)}")
    print("Possible solutions:")
    print("- Try again later (server might be blocking requests)")
    print("- Check if the website structure has changed")
    print("- Use alternative data sources like Yahoo Finance")

Error occurred: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.
Possible solutions:
- Try again later (server might be blocking requests)
- Check if the website structure has changed
- Use alternative data sources like Yahoo Finance


/tmp/ipykernel_143455/114999032.py:40: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tesla_revenue = pd.read_html(str(revenue_table))[0]


In [ ]:
pip install lxml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 3.5 MB/s eta 0:00:00m eta 0:00:010:01:01
